In [1]:
import pandas as pd
import numpy as np

In [16]:
Thermal = pd.read_csv('2032 ADS PCM V2.4.1 Public Data/Processed Data/ThermalGen.csv')
UCgen = pd.read_csv('output/UCGen.csv',header=None)

In [22]:
output0 = UCgen.iloc[0,:]

In [27]:
Thermal['output0'] = UCgen.iloc[0,:]

In [25]:
GPini = Thermal['InitialDispatch(MW)']
GRU = Thermal['RampUp Rate(MW/minute)']*5
GPmin = Thermal['IOMinCap(MW)']
U0 = Thermal['Initial Status']

In [29]:
subset_df = Thermal[Thermal['output0'] > 0]


In [54]:
sum(subset_df['output0'] - subset_df['InitialDispatch(MW)'] > subset_df['RampUp Rate(MW/minute)']*5)

264